<a href="https://colab.research.google.com/github/ooper-zz/sheet-music/blob/main/kareoke_gen_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install moviepy pydub librosa numpy

In [ ]:
!apt-get update
!apt-get install -y imagemagick ffmpeg libmagickwand-dev
!apt-get install fonts-dejavu-core

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,553 kB]
Get:14 http://archive.ubuntu.com/ubuntu ja

In [ ]:
import os

# Set the path to ImageMagick
os.environ['IMAGEMAGICK_BINARY'] = '/usr/bin/convert'

# List files in the current directory
print(os.listdir())


['.config', 'sample_data']


In [ ]:
!ls /usr/share/fonts/truetype/dejavu/

DejaVuSans-Bold.ttf	 DejaVuSansMono.ttf  DejaVuSerif-Bold.ttf
DejaVuSansMono-Bold.ttf  DejaVuSans.ttf      DejaVuSerif.ttf


In [ ]:
!pip install pillow


In [ ]:
from PIL import Image, ImageDraw, ImageFont
import moviepy.editor as mpy
import os
from google.colab import files
from pydub import AudioSegment

# Ensure ImageMagick is properly installed
!apt-get update
!apt-get install imagemagick ffmpeg libmagickwand-dev

# Upload the files
uploaded = files.upload()

# Automatically detect and assign file names
lyrics_file = None
mp3_file = None
cover_image_file = None

for filename in uploaded.keys():

     # the lyrics file has the github lyrics that is merged with the Chords AI lyrics.
    if filename.endswith('.txt'):
        lyrics_file = filename
    elif filename.endswith('.mp3'):
        mp3_file = filename
    elif filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        cover_image_file = filename

if not lyrics_file or not mp3_file or not cover_image_file:
    raise Exception("All three files (lyrics.txt, song.mp3, cover.jpg) must be uploaded!")

# Set the song name based on the mp3 file name (without the extension)
song_name = mp3_file.rsplit('.', 1)[0]

# Load the audio file using pydub
audio = AudioSegment.from_mp3(mp3_file)
audio_duration = len(audio) / 1000  # Duration in seconds

# Read the lyrics
with open(lyrics_file, 'r', encoding='utf-8') as f:
    lyrics = f.read().splitlines()

# Load the cover image
cover_image = mpy.ImageClip(cover_image_file)

# Create video clips for each line of lyrics using PIL to generate text images
clips = []
for i, line in enumerate(lyrics):
    # Create an image with the text
    img = Image.new('RGB', cover_image.size, color=(0, 0, 0))
    d = ImageDraw.Draw(img)
    # Load a font
    font = ImageFont.load_default()  # Or you can specify a path to a .ttf font file
    # Position the text in the center
    text_width, text_height = d.textsize(line, font=font)
    position = ((cover_image.size[0] - text_width) // 2, (cover_image.size[1] - text_height) // 2)
    d.text(position, line, font=font, fill=(255, 255, 255))

    # Save the image to a file
    text_img_path = f"text_image_{i}.png"
    img.save(text_img_path)

    # Load the image as an ImageClip
    txt_clip = mpy.ImageClip(text_img_path)
    clip_duration = audio_duration / len(lyrics) if len(lyrics) > 0 else 1  # Default to 1 second if no lyrics

    txt_clip = txt_clip.set_position('center').set_duration(clip_duration)

    # Combine with cover image
    composite_clip = mpy.CompositeVideoClip([cover_image, txt_clip]).set_duration(clip_duration)
    clips.append(composite_clip)




Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 128 kB in 3s (38.4 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it 